In [1]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from sklearn.linear_model import LinearRegression
from kmtr.datasets_and_metrics import MDS, mini_higgs, Higgs, relative_error, one_minus_auc
import time

# MDS

In [7]:
X_train, y_train, X_test, y_test = MDS()

In [ ]:
m = LinearRegression()
t0 = time.time()
m.fit(X_train, y_train)
t = time.time() - t0
y_pred = m.predict(X_test)
print(f"Error : {relative_error(y_pred, y_test):.5f}, Time : {t:.3f}")

Error : 0.00339, Time : 1.827


## Mini Higgs

In [2]:
X_train, y_train, X_test, y_test = mini_higgs()

In [5]:
m = LinearRegression()
t0 = time.time()
m.fit(X_train, y_train)
t = time.time() - t0
y_pred = m.predict(X_test)
print(f"Error : {one_minus_auc(y_pred, y_test):.5f}, Time : {t:.3f}")

Error : 0.31710, Time : 0.031
